
# Raster Priming with GDAL/OGR Command Line

**Brek Chiles, 2025**



### Purpose

This notebook provides a workflow with GDAL/OGR command line tools that create raster data inputs for the GeoPandas_Rasterio_Zonal_Stistics IPython Notebook.


In [14]:
# Set Dropbox Url and directory for input raster files
url = "https://www.dropbox.com/scl/fi/um1idvkj5taiinvsvbcgu/King_Co_2021_DTM_Crop.tif?rlkey=ziqgqu8lmhywilqiwcgtk9e1z&st=24nyyc36&dl=1"
raster_directory = r"C:\Users\brekc\OneDrive\Desktop\GIS_PROJECTS\Road_Slope_Analysis_Education_Hill\Python_Method\Inputs\DEM"

In [ ]:
# Set file paths
#  Raster
raster_input = r"C:\Users\brekc\OneDrive\Desktop\GIS_PROJECTS\Road_Slope_Analysis_Education_Hill\Python_Method\Inputs\DEM\King_Co_2021_DTM_Crop.tif"
raster_output = r"C:\Users\brekc\OneDrive\Desktop\GIS_PROJECTS\Road_Slope_Analysis_Education_Hill\Python_Method\Inputs\DEM\King_Co_2021_Ext.tif"
hillshade_output = r"C:\Users\brekc\OneDrive\Desktop\GIS_PROJECTS\Road_Slope_Analysis_Education_Hill\Python_Method\Inputs\DEM\King_Co_2021_SR.tif"
slope_output = r"C:\Users\brekc\OneDrive\Desktop\GIS_PROJECTS\Road_Slope_Analysis_Education_Hill\Python_Method\Inputs\DEM\King_Co_2021_Ext_Perc.tif"
# Vector/Geometry
redmond_neighborhoods = r"C:\Users\brekc\OneDrive\Desktop\GIS_PROJECTS\Road_Slope_Analysis_Education_Hill\Python_Method\Inputs\Boundary\Redmond_Neighborhoods.shp"
boundary = r"C:\Users\brekc\OneDrive\Desktop\GIS_PROJECTS\Road_Slope_Analysis_Education_Hill\Python_Method\Inputs\Boundary\Redmond_Study_Area.shp"


### Raster Extraction by Mask

The following cells will download the supplemental raster GeoTiff and extract data to the study area boundary.


In [15]:
!curl -L "$url"  -O --output-dir "$raster_directory"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100    17  100    17    0     0     30      0 --:--:-- --:--:-- --:--:--    30

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0  699M    0 98304    0     0  59967      0  3:23:55  0:00:01  3:23:54   98k
  5  699M    5 39.5M    0     0  15.0M      0  0:00:46  0:00:02  0:00:44 20.0M
 13  699M   13 92.0M    0     0  25.2M      0  0:00:27  0:00:03  0:00:24 30.9M
 21  699M   21  148M    0     0  31.6M      0  0:00:22  0:00:04  0:00:18 37.0M
 28  699M   28  200M    0     0  35.6M      0  0:00:19  0:00:05  0:00:14 40.3M
 36  699M   36  256M    0     0  38.4M      0  0:00:18  0:00:06  0:00:12 51.0M
 44  699M   44  311M    0     0  40.7M      0  0:00:17  0:00:07  0:00:10 54.3M
 52  699M   52  366M    0     0  42.3M      0  0:0


Inspect raster metadata with gdalinfo.


In [16]:
!gdalinfo $raster_input

Driver: GTiff/GeoTIFF
Files: C:\Users\brekc\OneDrive\Desktop\GIS_PROJECTS\Road_Slope_Analysis_Education_Hill\Python_Method\Inputs\DEM\King_Co_2021_DTM_Crop.tif
Size is 12148, 15098
Coordinate System is:
PROJCRS["NAD83(HARN) / Washington North (ftUS)",
    BASEGEOGCRS["NAD83(HARN)",
        DATUM["NAD83 (High Accuracy Reference Network)",
            ELLIPSOID["GRS 1980",6378137,298.257222101,
                LENGTHUNIT["metre",1]]],
        PRIMEM["Greenwich",0,
            ANGLEUNIT["degree",0.0174532925199433]],
        ID["EPSG",4152]],
    CONVERSION["SPCS83 Washington North zone (US survey foot)",
        METHOD["Lambert Conic Conformal (2SP)",
            ID["EPSG",9802]],
        PARAMETER["Latitude of false origin",47,
            ANGLEUNIT["degree",0.0174532925199433],
            ID["EPSG",8821]],
        PARAMETER["Longitude of false origin",-120.833333333333,
            ANGLEUNIT["degree",0.0174532925199433],
            ID["EPSG",8822]],
        PARAMETER["Latitude of 1st


Inspect vector or geometric metadata with ogrlinfo. Check out Ogr2Ogr.


In [17]:
!ogrinfo -al $redmond_neighborhoods

INFO: Open of `C:\Users\brekc\OneDrive\Desktop\GIS_PROJECTS\Road_Slope_Analysis_Education_Hill\Python_Method\Inputs\Boundary\Redmond_Neighborhoods.shp'
      using driver `ESRI Shapefile' successful.

Layer name: Redmond_Neighborhoods
Metadata:
  DBF_DATE_LAST_UPDATE=2025-07-10
Geometry: Polygon
Feature Count: 6
Extent: (1315310.500206, 240556.350049) - (1333529.220330, 263200.600036)
Layer SRS WKT:
PROJCRS["NAD83(HARN) / Washington North (ftUS)",
    BASEGEOGCRS["NAD83(HARN)",
        DATUM["NAD83 (High Accuracy Reference Network)",
            ELLIPSOID["GRS 1980",6378137,298.257222101,
                LENGTHUNIT["metre",1]]],
        PRIMEM["Greenwich",0,
            ANGLEUNIT["degree",0.0174532925199433]],
        ID["EPSG",4152]],
    CONVERSION["SPCS83 Washington North zone (US survey foot)",
        METHOD["Lambert Conic Conformal (2SP)",
            ID["EPSG",9802]],
        PARAMETER["Latitude of false origin",47,
            ANGLEUNIT["degree",0.0174532925199433],
           

Dissolve redmond_neighborhoods to make boundary

In [18]:
!ogr2ogr -f "ESRI Shapefile" $boundary $redmond_neighborhoods -dialect sqlite -sql "SELECT ST_Union(geometry), NAME FROM 'Redmond_Neighborhoods' GROUP BY CreatedBy"


Use gdalwarp to extract raster data to a vector or geometry file


In [19]:
!gdalwarp -cutline $boundary -crop_to_cutline $raster_input $raster_output -overwrite

Creating output file that is 12147P x 15097L.
Processing C:\Users\brekc\OneDrive\Desktop\GIS_PROJECTS\Road_Slope_Analysis_Education_Hill\Python_Method\Inputs\DEM\King_Co_2021_DTM_Crop.tif [1/1] : 0Using internal nodata values (e.g. -3.40282e+38) for image C:\Users\brekc\OneDrive\Desktop\GIS_PROJECTS\Road_Slope_Analysis_Education_Hill\Python_Method\Inputs\DEM\King_Co_2021_DTM_Crop.tif.
Copying nodata values from source C:\Users\brekc\OneDrive\Desktop\GIS_PROJECTS\Road_Slope_Analysis_Education_Hill\Python_Method\Inputs\DEM\King_Co_2021_DTM_Crop.tif to destination C:\Users\brekc\OneDrive\Desktop\GIS_PROJECTS\Road_Slope_Analysis_Education_Hill\Python_Method\Inputs\DEM\King_Co_2021_Ext.tif.
...10...20...30...40...50...60...70...80...90...100 - done.



### Raster Processing with gdaldem

GDAL/OGR can analyze DEM data with gdaldem.




Create a Hillshade/Shaded Relief map


In [20]:
!gdaldem hillshade -az 275 $raster_input $hillshade_output

0...10...20...30...40...50...60...70...80...90...100 - done.



Create a slope map measured in percent (-p)


In [21]:
!gdaldem slope -p $raster_output $slope_output

0...10...20...30...40...50...60...70...80...90...100 - done.
